# Kohya SDXL Fine‑Tune Notebook
This Colab notebook fine‑tunes an SDXL model with [kohya‑ss/sd‑scripts](https://github.com/kohya-ss/sd-scripts).
_Automatically instrumented for detailed debug logging as requested by Mikey._

### Clone kohya‑ss/sd‑scripts

In [ ]:
# Install required libraries
%cd /content
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts
!pip install -r requirements.txt
print('✅ kohya-ss repo cloned and requirements installed.')


## 2. Prepare Google Drive (optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#  Set Training Paths


In [ ]:
import os# Define pathsbase_model_path = "/content/drive/MyDrive/models/homosimileXLPony_v20ILXL.safetensors"diff_model_path = "/content/drive/MyDrive/models/homosimileXLPony_v20ILXL_diffusers"dataset_dir = "/content/drive/MyDrive/Loras/kmk/dataset"output_dir = "/content/drive/MyDrive/Loras/kmk/output"logging_dir = "/content/drive/MyDrive/Loras/kmk/logs"cache_latents_dir = "/content/drive/MyDrive/Loras/kmk/cache_latents"# Make sure all folders existos.makedirs(output_dir, exist_ok=True)os.makedirs(logging_dir, exist_ok=True)os.makedirs(cache_latents_dir, exist_ok=True)

In [ ]:
# Convert checkpoint to Diffusers format
import osfrom library import sdxl_model_utilckpt = base_model_pathout_dir = diff_model_pathos.makedirs(out_dir, exist_ok=True)print('Converting checkpoint to Diffusers format...')text_encoder1, text_encoder2, vae, unet, _, _ = sdxl_model_util.load_models_from_sdxl_checkpoint(    sdxl_model_util.MODEL_VERSION_SDXL_BASE_V1_0, ckpt, "cpu")sdxl_model_util.save_diffusers_checkpoint(out_dir, text_encoder1, text_encoder2, unet, None, vae, use_safetensors=True)print('Saved Diffusers model to', out_dir)


# ✅  Prepare latents

In [ ]:
# Setup Accelerate default configuration
import os
config_path = '/content/accelerate/default_config.yaml'
os.makedirs('/content/accelerate', exist_ok=True)
!accelerate config default --config_file $config_path
os.environ['ACCELERATE_CONFIG_FILE'] = config_path
print(f'Accelerate config written to {config_path}')


In [ ]:
# Optional speed boost: Cache latents
!ACCELERATE_CONFIG_FILE=$config_path accelerate launch --num_cpu_threads_per_process 1 tools/cache_latents.py --sdxl --pretrained_model_name_or_path="$diff_model_path" --train_data_dir="$dataset_dir" --resolution="1024,1024" --vae_batch_size=1 --cache_latents_to_disk


#✅ start training


In [ ]:
# Start training
!ACCELERATE_CONFIG_FILE=$config_path accelerate launch --num_cpu_threads_per_process 1 sdxl_train.py --pretrained_model_name_or_path="$diff_model_path" --train_data_dir="$dataset_dir" --resolution="1024,1024" --output_dir="$output_dir" --logging_dir="$logging_dir" --output_name="kmk_sdxl_finetuned" --learning_rate=1e-4 --lr_scheduler="cosine_with_restarts" --train_batch_size=1 --max_train_steps=1500 --save_every_n_steps=500 --mixed_precision="fp16" --cache_latents --cache_latents_to_disk --cache_text_encoder_outputs --vae_batch_size=1 --caption_extension=".txt" --caption_dropout_rate=0.15 --xformers


#✅ Check outputs

In [ ]:
# Check output folder contents
import os

for f in os.listdir(output_dir):
    print(f)
